In [ ]:
import numpy as np
import pickle

In [4]:
class Qlearning:
    # learn_r -> learning rate      exp_r -> experimenting rate (how often to randomly choose)
    def __init__(self, learn_r=0.1, exp_r=0.3):  

        self.player_Q_Values = {}  
        
        # key: [(player_val, up_card, ace)][action] = value
        # initialise Q values | (12-21) x (1-10) x (True, False) x (1, 0) 400 in total
            # This essentially represents each possible action in the game. Each combination of
                # player sum (hard and soft)/ dealer up card with 1 and 0 to represent the choice for that combination
        
        for i in range(12, 22):
            for j in range(1, 12):
                for k in [True, False]:
                    self.player_Q_Values[(i, j, k)] = {}
                    for a in [1, 0]:
                        self.player_Q_Values[(i, j, k)][a] = 0
        
        
        self.state = (0, 0, False)  # default state
        self.actions = [1, 0]  # 1 -> hit, 0 -> stand
        self.player_state_action = []
        self.end = False
        self.learn_r = learn_r
        self.exp_r = exp_r

    # Same logic here as in MC
    @staticmethod
    def giveCard():
        cards = [1,2,3,4,5,6,7,8,9,10,10,10,10]
        return np.random.choice(cards)
    
    def dealerLogic(self, val, ace):
        if val > 21:
            if ace:
                val -= 10
                ace = False
            else:
                return val, ace, True    
    
        # Assuming dealer stands on hard or soft 17, can be changed later
        if val >= 17 and not ace:
            return val, ace, True
        return self.aceHelper(val,ace)
    

    def chooseAction(self):

        # Always hit if val <= 11. Can change for testing

        current_val = self.state[0]
        if current_val <= 11:
            return 1
        
        # Here we balance experimentation vs explotiation
            # This means exp_r precent of the time, hit or stay is chosen at random
                # Otherwise, we rely on action that gains the besr reward
        
        # 'Decide' if a random choice will be made 
        if np.random.uniform(0, 1) <= self.exp_r:
            decision = np.random.choice(self.actions)
        else:
            # Greedy action
            v = float('-inf')
            decision = 0
            for a in self.player_Q_Values[self.state]:
                if self.player_Q_Values[self.state][a] > v:
                    decision = a
                    v = self.player_Q_Values[self.state][a]
        return decision

# Take in action, update to next state, and determine if game is over
def playerNxtState(self, action):
        current_val, up_card, ace = self.state      #unpack tuple
        
        if current_val > 21 and ace:
            #if ace:
            current_val -= 10
            ace = False
            #else:
                # should not reach here
                #self.end = True
                #self.state = (current_val, up_card, ace)
                #return
        if action:
            card = self.giveCard()
            if card == 1:
                if current_val <= 10:
                    current_val += 11
                    ace = True
                else:
                    current_val += 1
            else:
                current_val += card
        else:
            # Player stands
            self.end = True
        
        if current_val > 21:
            self.end = True
        self.state = (current_val, up_card, ace)